# Split-Apply-Combine Method

Categorizing a dataset and applying a function to each group, whether an aggregation or transformation, is often critical component of a data analysis workflow. After load, merging and preparing a dataset, you may need to compute group statistics possibly pivot tables for reporting or visualization purposes. Pandas provides a flexible groupby interface enabling you to slice, dice and summarize datasets in a natural way.

Probably one of the reson for SQL and relational database popularity is the ease with which data can be joined, filtered, transformed and aggregated. However, query languages like SQL are somewhat constrained in the kind of group operations that can be performed. This is where python expressiveness stands out. We can perform quite complex group operations by utilizing any function that accepts a pandas object or numpy array.

Hadley Wickham, an author of many popular R packages, coined the term split-apply-combine for describing group operations.In the first stage of the process, data contained in a pandas object whether a Series, DataFrame or otherwise, is split into groups based on one or more criteria (or key) that you provide.

The splitting can be grouped on its rows (axis=0) or its columns (axis=1). Once this is done, a funcntion is applied to each group, producing a new value. Finally, the results of those function applications are combined into a result object. The form of the resulting object will usually depend on what's being done to the data.

<img src="assets/images/general-split-apply-combine.svg" width="700" />

Each grouping key can take many forms, and the keys do not have to be all of the same type:

- a list or array of values that is the same length as the axis being grouped
- a value indicating a column name in a DataFrame
- a dict or series giving correspondence between values on the axis being grouped and group names
- a function to be invoked on the axis index or the individual labels in the index

In [1]:
import pandas as pd
import numpy as np

## Group By

In [2]:
product_sheet = pd.DataFrame({
    'product': np.random.choice(['shirt','shoes', 'pants'], 50),
    'color': np.random.choice(['black', 'red', 'green', 'yellow'], 50),
    'manufacturer': np.random.choice(['company 1', 'company 2', 'company 3'], 50),
    'price': np.random.randint(30, 100, 50),
    'discount': np.random.randint(10, 25, 50)
})

In [3]:
product_sheet.head()

,product,color,manufacturer,price,discount
0,shirt,red,company 1,45,17
1,pants,green,company 2,45,18
2,shoes,red,company 2,92,16
3,shoes,red,company 2,47,20
4,pants,red,company 3,76,23


Now, suppose you want to compute the average price of products per manufacturer. One way to do it is to call groupby on `product` and `manufacturer` which will return a GroupBy object. It has not actually computed anything yet except for some intermediate data. The idea is that this object has all information needed to then apply some operation to each groups.

So, for example we can apply the `mean()` function to the group to get the average per product per manufacturer

In [4]:
grouped = product_sheet.groupby(['manufacturer', 'product'])
grouped.mean()['price']

manufacturer  product
company 1     pants      61.666667
              shirt      51.444444
              shoes      59.285714
company 2     pants      65.833333
              shirt      48.000000
              shoes      70.375000
company 3     pants      55.500000
              shirt      68.000000
              shoes      76.666667
Name: price, dtype: float64

We can also display this computed value into a more intuitive form by calling `unstack()`. 

The take away is, the data (a Series) has been aggregated according to the group key, producing a new Series that is now index by unique values used to group the original dataset, in this case, (manufacturer, product).

In [5]:
grouped.mean()['price'].unstack()

product,pants,shirt,shoes
manufacturer,,,
company 1,61.666667,51.444444,59.285714
company 2,65.833333,48.000000,70.375000
company 3,55.500000,68.000000,76.666667


### Iterating over Groups

The GroupBy object supports iteration, generating a sequence of 2-tuples containing the group name along with the chunk of data.

For example, our data was grouped by manufacturer and product so we have a tuple as a key with the following values

In [6]:
for key, group in grouped:
  print(key)

('company 1', 'pants')
('company 1', 'shirt')
('company 1', 'shoes')
('company 2', 'pants')
('company 2', 'shirt')
('company 2', 'shoes')
('company 3', 'pants')
('company 3', 'shirt')
('company 3', 'shoes')


### Grouping by Dict and Series
Grouping information may exist in a form other than an array. consider the following DataFrame

In [7]:
people = pd.DataFrame(
  np.random.randn(5,5), 
  columns=['a', 'b', 'c', 'd', 'e'], 
  index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis']
)
people.iloc[2:3, [1,2]] = np.nan
people

,a,b,c,d,e
Joe,-0.291731,-1.488735,-0.495062,1.177757,0.373863
Steve,-0.609132,-1.363981,-0.015473,0.382132,0.759272
Wes,-0.574761,NaN,NaN,-1.707388,0.608860
Jim,-0.204217,0.907058,-0.749650,0.977011,1.336234
Travis,-2.198780,0.518779,0.868977,0.622631,-0.049697


Now, suppose I have a correspondence for the columns and want to sum together the columns by group:


In [8]:
mapping = {
  'a': 'red', 
  'b': 'red', 
  'c': 'blue', 
  'd': 'blue', 
  'e': 'red', 
  'f': 'orange' 
}

We could construct an array from this dict to pass to groupby, but instead we can pass the dict like so:

In [9]:
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,0.682694,-1.406604
Steve,0.366659,-1.213841
Wes,-1.707388,0.034099
Jim,0.227361,2.039074
Travis,1.491608,-1.729698


Unused grouping keys will be ignored, in this case `f`.

The same functionality holds for Series, which can be viiewed as a fixed size mapping:

In [10]:
pd.Series(mapping)

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [11]:
people.groupby(mapping, axis=1).sum()

,blue,red
Joe,0.682694,-1.406604
Steve,0.366659,-1.213841
Wes,-1.707388,0.034099
Jim,0.227361,2.039074
Travis,1.491608,-1.729698


### Grouping with Functions

Mapping with a dict or a series is very easy. However, it operates under the assumption that you have a fixed map for your data. Sometimes, this isn't the case.

Suppose you wanted to group by the length of names; while you could compute an array of string lengths, it's simpler to just apss the len function.

In [12]:
for length, group in people.groupby(len):
  print(f'Name length: {length}')
  display(group)

Name length: 3


,a,b,c,d,e
Joe,-0.291731,-1.488735,-0.495062,1.177757,0.373863
Wes,-0.574761,NaN,NaN,-1.707388,0.608860
Jim,-0.204217,0.907058,-0.749650,0.977011,1.336234


Name length: 5


,a,b,c,d,e
Steve,-0.609132,-1.363981,-0.015473,0.382132,0.759272


Name length: 6


,a,b,c,d,e
Travis,-2.19878,0.518779,0.868977,0.622631,-0.049697


Further, mixing functions and dict is not a problem as everething is converted into lists internally

In [13]:
key_team = ['Team 1', 'Team 1', 'Team 2', 'Team 2', 'Team 1']

for key, group in people.groupby([len, key_team]):
  length, team = key
  print(f'Name length: {length}, Team: {team}')
  display(group)

Name length: 3, Team: Team 1


,a,b,c,d,e
Joe,-0.291731,-1.488735,-0.495062,1.177757,0.373863


Name length: 3, Team: Team 2


,a,b,c,d,e
Wes,-0.574761,NaN,NaN,-1.707388,0.608860
Jim,-0.204217,0.907058,-0.74965,0.977011,1.336234


Name length: 5, Team: Team 1


,a,b,c,d,e
Steve,-0.609132,-1.363981,-0.015473,0.382132,0.759272


Name length: 6, Team: Team 1


,a,b,c,d,e
Travis,-2.19878,0.518779,0.868977,0.622631,-0.049697


In [14]:
key_school = {
    'Joe': 'University C', 
    'Steve': 'University B',
    'Wes': 'University C',
    'Jim': 'University B',
    'Travis': 'University A',
}

for key, group in people.groupby([len, key_school]):
  length, team = key
  print(f'Name length: {length}, Team: {team}')
  display(group)

Name length: 3, Team: University B


,a,b,c,d,e
Jim,-0.204217,0.907058,-0.74965,0.977011,1.336234


Name length: 3, Team: University C


,a,b,c,d,e
Joe,-0.291731,-1.488735,-0.495062,1.177757,0.373863
Wes,-0.574761,NaN,NaN,-1.707388,0.608860


Name length: 5, Team: University B


,a,b,c,d,e
Steve,-0.609132,-1.363981,-0.015473,0.382132,0.759272


Name length: 6, Team: University A


,a,b,c,d,e
Travis,-2.19878,0.518779,0.868977,0.622631,-0.049697


## Data Aggregation

Aggregation refers to any data transformation that produces scalar values from arrays like mean, count, min, sum, etc. However, we are not limited to these methods.

Here is a list of the optmized GroupBy methods

- count: Numer of non-NA values in group
- sum: Sum of all non-NA values
- mean: Mean of all non-NA values
- median: Arithmetic median of all non-NA values
- std, var: Unbiased(n-1) standard deviation and variance
- min, max: Minimum and maximum of non-NA values
- prod: Product of non-NA values
- first, last: First and last non-NA values

You can also use your own custom aggreagte functions and additionally call any method that is also defined on the grouped object. For example, quartile computes the sample quartiles of a Series or a DataFrame's columns. While quartile is not explicitly implemented for GroupBy object, it is a Series method and thus available for use.

To use your own aggregation functions, pass any function that aggreagates an array to the aggreagate or agg method:

In [15]:
def peak_to_peak(arr):
  return arr.max() - arr.min()

grouped.agg(peak_to_peak)

discount  price
manufacturer product                 
company 1    pants           5     37
             shirt          11     63
             shoes          14     52
company 2    pants          13     56
             shirt           0      0
             shoes           9     56
company 3    pants          11     58
             shirt           8     51
             shoes           8     45

### Column-wise and Multiple Function Application

Aggregating a Series or all of the columns of a DataFrame is a matter of using aggregate with desired function or calling a method like mean or std. However, you may want to aggreagte using a different function depending on the coolumn or multiple functions at once. Fortunately, this is very easy to do.

In [16]:
import seaborn as sns

restaurant = sns.load_dataset('tips')

In [17]:
restaurant['tip_pct'] = restaurant['tip'] / restaurant['total_bill']

In [18]:
grouped = restaurant.groupby(['day', 'smoker'])
grouped["tip_pct"].agg('mean')

day   smoker
Thur  Yes       0.163863
      No        0.160298
Fri   Yes       0.174783
      No        0.151650
Sat   Yes       0.147906
      No        0.158048
Sun   Yes       0.187250
      No        0.160113
Name: tip_pct, dtype: float64

In [19]:
grouped["tip_pct"].agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Thur Yes     0.163863  0.039389      0.151240
     No      0.160298  0.038774      0.193350
Fri  Yes     0.174783  0.051293      0.159925
     No      0.151650  0.028123      0.067349
Sat  Yes     0.147906  0.061375      0.290095
     No      0.158048  0.039767      0.235193
Sun  Yes     0.187250  0.154134      0.644685
     No      0.160113  0.042347      0.193226

You don't need to accept the names that GroupBy gives to columns; notably for lamda functions that have the name '<lambda>', which makes them hard to identify. Thus, if you pass a list of (name, function) tuples, the first element of each tuple will be used as the DataFrame column name.

In [20]:
grouped["tip_pct"].agg([('func1', 'mean'), ('func2', 'std'), ('func3', peak_to_peak)])

func1     func2     func3
day  smoker                              
Thur Yes     0.163863  0.039389  0.151240
     No      0.160298  0.038774  0.193350
Fri  Yes     0.174783  0.051293  0.159925
     No      0.151650  0.028123  0.067349
Sat  Yes     0.147906  0.061375  0.290095
     No      0.158048  0.039767  0.235193
Sun  Yes     0.187250  0.154134  0.644685
     No      0.160113  0.042347  0.193226

With a DaraFrame, you have more options, as you can specify which columns or different functions per column. To start, suppose we wanted to compute the same three statistiics for the `tip_pct` and the `total_bill` columns:

In [21]:
result = grouped["tip_pct", "total_bill"].agg(['mean', 'std', peak_to_peak])
result

tip_pct                        total_bill             \
                 mean       std peak_to_peak       mean        std   
day  smoker                                                          
Thur Yes     0.163863  0.039389     0.151240  19.190588   8.355149   
     No      0.160298  0.038774     0.193350  17.113111   7.721728   
Fri  Yes     0.174783  0.051293     0.159925  16.813333   9.086388   
     No      0.151650  0.028123     0.067349  18.420000   5.059282   
Sat  Yes     0.147906  0.061375     0.290095  21.276667  10.069138   
     No      0.158048  0.039767     0.235193  19.661778   8.939181   
Sun  Yes     0.187250  0.154134     0.644685  24.120000  10.442511   
     No      0.160113  0.042347     0.193226  20.506667   8.130189   

                          
            peak_to_peak  
day  smoker               
Thur Yes           32.77  
     No            33.68  
Fri  Yes           34.42  
     No            10.29  
Sat  Yes           47.74  
     No            41.08  
Sun  Yes           38.10  
     No            39.40

As you can see, the resulting DataFrame has hierarchical columns, the same as you would get aggregating each column separately and using concat to glue the results together using the column names as the keys argument:

In [22]:
result["tip_pct"]

mean       std  peak_to_peak
day  smoker                                  
Thur Yes     0.163863  0.039389      0.151240
     No      0.160298  0.038774      0.193350
Fri  Yes     0.174783  0.051293      0.159925
     No      0.151650  0.028123      0.067349
Sat  Yes     0.147906  0.061375      0.290095
     No      0.158048  0.039767      0.235193
Sun  Yes     0.187250  0.154134      0.644685
     No      0.160113  0.042347      0.193226

In [23]:
result["total_bill"]

mean        std  peak_to_peak
day  smoker                                    
Thur Yes     19.190588   8.355149         32.77
     No      17.113111   7.721728         33.68
Fri  Yes     16.813333   9.086388         34.42
     No      18.420000   5.059282         10.29
Sat  Yes     21.276667  10.069138         47.74
     No      19.661778   8.939181         41.08
Sun  Yes     24.120000  10.442511         38.10
     No      20.506667   8.130189         39.40

Now, suppose you want to apply potentially different functions to one or more of the columns. To do thism pass a dict to agg that contains a mapping of the column names to any of the function specifications listed so far:

In [24]:
grouped.agg({ 'tip': np.max, 'size': 'sum', 'tip_pct': ['min', 'max', 'mean', 'std'] })

tip size   tip_pct                              
              amax  sum       min       max      mean       std
day  smoker                                                    
Thur Yes      5.00   40  0.090014  0.241255  0.163863  0.039389
     No       6.70  112  0.072961  0.266312  0.160298  0.038774
Fri  Yes      4.73   31  0.103555  0.263480  0.174783  0.051293
     No       3.50    9  0.120385  0.187735  0.151650  0.028123
Sat  Yes     10.00  104  0.035638  0.325733  0.147906  0.061375
     No       9.00  115  0.056797  0.291990  0.158048  0.039767
Sun  Yes      6.50   49  0.065660  0.710345  0.187250  0.154134
     No       6.00  167  0.059447  0.252672  0.160113  0.042347

In [25]:
restaurant.groupby(['day', 'smoker'], as_index=False).agg({ 'tip': np.max, 'size': 'sum', 'tip_pct': ['min', 'max', 'mean', 'std'] })

day smoker    tip size   tip_pct                              
                 amax  sum       min       max      mean       std
0  Thur    Yes   5.00   40  0.090014  0.241255  0.163863  0.039389
1  Thur     No   6.70  112  0.072961  0.266312  0.160298  0.038774
2   Fri    Yes   4.73   31  0.103555  0.263480  0.174783  0.051293
3   Fri     No   3.50    9  0.120385  0.187735  0.151650  0.028123
4   Sat    Yes  10.00  104  0.035638  0.325733  0.147906  0.061375
5   Sat     No   9.00  115  0.056797  0.291990  0.158048  0.039767
6   Sun    Yes   6.50   49  0.065660  0.710345  0.187250  0.154134
7   Sun     No   6.00  167  0.059447  0.252672  0.160113  0.042347

## General Split-Apply-Combine

In [26]:
def top(df, n=5, column='tip_pct'):
  return df.sort_values(by=column)[-n:]